In [3]:
#Remove conflicting packages from the Kaggle base environment.¶
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 11.5 MB/s eta 0:00:00


In [4]:
import numpy as np 
import pandas as pd 
import os
import operator
import re # Import regex for parsing
import json
import io
import sys



from langchain_core.messages import BaseMessage, HumanMessage, AIMessage # Import AIMessage
# from langchain_openai import ChatOpenAI # Replace with your desired LLM provider
from langgraph.graph import StateGraph, END
# Kaggle and Google AI
from kaggle_secrets import UserSecretsClient
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai
from google.genai import types
from google.api_core import retry

# IPython Display
from IPython.display import Markdown, Image, display

# PDF Processing
import pypdf

# ChromaDB
#import chromadb
#from chromadb import Documents, EmbeddingFunction, Embeddings


from typing import TypedDict, Annotated, Optional, Literal, List, Dict, Any
from typing_extensions import TypedDict # 
from langchain_core.messages import BaseMessage 
from contextlib import redirect_stdout

# Langchain and Langgraph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage # Ensure these are imported


# Pretty Print
from pprint import pprint
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")
train, test, submission = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv'),pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv'),pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


# **🎯🚀We will defined langgraph who will coordinate our data analyst and data scienctist nodes🎯🚀**
# **Lets build a 🤖 herarchical 🤖 architecture**
# 1. supervisor
# 2. data scientist
# 3. data analyst
# 4. outputs to Human interpreter 

In [5]:
# Using simple dicts for messages initially for clarity, but BaseMessage[] is better practice# Example: messages: Annotated[list[BaseMessage], add_messages]
# Using simple dicts for messages initially for clarity, but BaseMessage[] is better practice# Example: messages: Annotated[list[BaseMessage], add_messages]
# --- State Definition ---
class GraphState(TypedDict):
    """
    🗂️ Central state definition for the CALOR-IA multi-agent workflow.
    Reflects using DataFrame variable names instead of file paths.
    """
    messages: Annotated[List[BaseMessage], operator.add]
    supervisor_tasks: Optional[List[str]]
    current_task_description: Optional[str]
    analyst_output: Optional[Dict[str, Any]]
    analyst_llm_output: Optional[Dict[str, Any]]
    scientist_llm_output: Optional[Dict[str, Any]]
    scientist_output: Optional[Dict[str, Any]]
    code_final: Optional[Dict[str, Any]]
    processed_train_name: Optional[str]     
    processed_test_name: Optional[str]
    error: Optional[str]
    next_agent: Optional[Literal["DataAnalyst",  "DataScientist", "DataAnalystIA","DataScientistIA", "Supervisor", "code_compiler_AI","HumanInterpreter", "__end__"]] 
    interpreter_finished : bool
    final_answer_generated: bool # Added for clarity


In [6]:

CALOR_IA_DATA_ANALYST_SYSINT = (
    "system",
    """
    🤖 You are CALOR-IA Data Analyst Bot.

    🎯 Mission
    1️⃣ Execute the data analysis task provided in the `current_task_description` from the Supervisor.
    2️⃣ Write clean, runnable Python 🐍 code for data loading, cleaning, exploration, and visualization (using pandas, matplotlib, seaborn, etc.).
    3️⃣ Analyze data (potentially loaded from `intermediate_data_path` if provided) and extract valuable insights.
    4️⃣ Prepare data for the Data Scientist if requested (e.g., creating features, splitting data) and potentially save it, updating `intermediate_data_path`.
    5️⃣ Place your results (code, summary of findings, paths to saved plots or data) into the `analyst_output` dictionary in the graph state.
    6️⃣ Collaborate with CALOR_IA_DATA_SCIENTIST_SYSINT via the Supervisor by providing necessary data artifacts and insights.

    📝 Style & Rules
    - Focus solely on the task in `current_task_description`.
    - Separate numerical and categorical data for better handling 
    - Output results clearly structured within the `analyst_output` dictionary (e.g., `{"code": "...", "summary": "...", "plot_path": "/path/to/plot.png", "data_preview": "..."}`).
    - Generate runnable Python code within markdown code blocks (```python ... ```).
    - Explain your code and findings concisely.
    - If you save data or plots, mention the path clearly in your output summary.
    - Use matplotlib/seaborn for plots.
    - Stick to data analysis/preparation; defer modeling to the Scientist.
    - Return the task to supervisor in json format for better understanding and the python code used to develop the task
    - Return Just the python code
    
    Let’s crunch some data! ✨
    """
)

CALOR_IA_DATA_SCIENTIST_SYSINT = (
    "system",
    """
    🤖 You are CALOR-IA Data Scientist Bot.

    🎯 Mission
    1️⃣ Execute the machine learning task provided in the `current_task_description` from the Supervisor.
    2️⃣ Use data provided (potentially loaded from `intermediate_data_path` prepared by the Analyst) to build, train, and evaluate ML models (using scikit-learn, TensorFlow, PyTorch, etc.).
    3️⃣ Perform feature engineering if required and not already done by the Analyst.
    4️⃣ Generate predictions on test data as requested.
    5️⃣ Place your results (model summary, performance metrics, paths to saved models or predictions) into the `scientist_output` dictionary in the graph state.
    6️⃣ Collaborate with CALOR_IA_DATA_ANALYST_SYSINT via the Supervisor by requesting specific data views or providing model insights.

    📝 Style & Rules
    - Focus solely on the task in `current_task_description`.
    - Output results clearly structured within the `scientist_output` dictionary (e.g., `{"model_description": "...", "metrics": {"accuracy": 0.95, ...}, "predictions_path": "/path/to/preds.csv", "code": "..."}`).
    - Generate runnable Python code for model definition, training, and prediction within markdown code blocks (```python ... ```).
    - Explain your model choices, training procedures, and evaluation metrics step-by-step.
    - If you save models or predictions, mention the path clearly in your output summary.
    - Aim for models that generalize well.
    - Stick to machine learning tasks; defer data prep/exploration to the Analyst unless specified.
    - Return the task to supervisor in json format for better understanding and the python code used to develop the task
    - All models must be scored in RMSLE=n1​i=1∑n​(log(1+y^​i​)−log(1+yi​))2
    - You will run the models on test data
    Let’s build some high-performing models! ✨
    """
)

CALOR_IA_CODE_COMPILER_SYSINT = (
    "system",
    f"""
    🤖 This function implements the CALOR-IA Code Compiler Bot, which extracts and fixes code fragments from the workflow output. The bot:

    🎯 Mission:
    use following inputs :
    train, test, submission = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv'),pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv'),pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')
    with the purpose of 
    1️⃣. Searches for code blocks enclosed in triple backticks
    2️⃣. Falls back to looking for common code patterns if no blocks are found
    3️⃣. Generates a complete, fixed script with:
       - Proper data loading
       - Correct variable references
       - Complete model training code
       - Prediction generation
       - Feature importance visualization
       - Output file generation
    
    📝 To use this bot in your workflow:
    
    1. Import the function
    2. Pass it the supervisor output text
    3. Use the returned string as your complete, ready-to-run Python script

The script includes all necessary error handling and produces a standardized output format.
"""
)

In [24]:
def supervisor_node(state: GraphState) -> Dict[str, Any]:
    """
    Central decision-making node. Routes tasks to appropriate agents
    or ends the workflow based on the current state.
    Now routes to DataAnalyst_AI AFTER HumanInterpreter.
    """
    print("\n--- SUPERVISOR ---")
    messages = state.get("messages", [])
    last_message = messages[-1] if messages else None
    print(f"Supervisor reviewing state. Last message type: {type(last_message).__name__ if last_message else 'None'}")
    if hasattr(last_message, 'content'):
         content_display = str(last_message.content)
         print(f"Supervisor received content: {content_display[:200]}{'...' if len(content_display) > 200 else ''}")

    # --- Retrieve outputs from potential sources ---
    analyst_orig_output = state.get('analyst_output')
    scientist_output_data = state.get('scientist_output')
    analyst_llm_output = state.get('analyst_llm_output')
    scientist_llm_output = state.get('scientist_llm_output')
    code = state.get('code_final')
    interpreter_finished = state.get('interpreter_finished', False)
    final_answer_generated = state.get('final_answer_generated', False)
    initial_train_name = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
    initial_test_name = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')
    tasks = state.get('supervisor_tasks', [])
    error_flag = state.get('error')

    print(f"Original Analyst output available: {analyst_orig_output is not None}")
    print(f"Scientist output available: {scientist_output_data is not None}")
    print(f"Interpreter finished signal: {interpreter_finished}")
    print(f"Final answer generated flag: {final_answer_generated}")
    
    next_agent = None
    task = None
    new_message_content = None
    updated_state_for_return = {}

    # --- Decision Logic (Ordered by Priority) ---
    
    # 1. Check for final answer signal - highest priority
    if final_answer_generated:
        print("Supervisor: Final answer signal received. Ending workflow.")
        print(code)
        next_agent = "__end__"
        # No new message needed
    
    # 2. Check if Scientist AI has completed review
    elif scientist_llm_output is not None and not final_answer_generated:
        print("Supervisor: Scientist AI review complete. Final answer should be ready.")
        # Force end the workflow since we've gone through all steps
        next_agent = "code_compiler_AI"
        new_message_content = "Workflow complete. Final script and reviews have been generated."
        updated_state_for_return['final_answer_generated'] = True  # Force set this flag
    
    # 3. Check if Analyst AI has finished its review - Next step is Scientist AI
    elif analyst_llm_output is not None and scientist_llm_output is None:
        print("Supervisor: Analyst AI review complete. Routing to Scientist AI for review.")
        next_agent = "DataScientist_AI"
        new_message_content = "Analyst AI review complete. Routing compiled script to Data Scientist AI for review."
    
    # 4. Check if Interpreter has finished - Start AI review chain
    elif interpreter_finished and analyst_llm_output is None:
        # Reset the flag to prevent re-triggering
        updated_state_for_return['interpreter_finished'] = False
        print("Supervisor: Interpreter finished compiling script. Routing to Data Analyst AI for review.")
        next_agent = "DataAnalyst_AI"
        new_message_content = "Interpreter step complete. Routing compiled script for AI review."
    
    # 5. Check for error condition
    elif error_flag is not None:
        print(f"Supervisor: Error detected in state: {error_flag}. Routing to Human Interpreter for review.")
        next_agent = "HumanInterpreter"
    
    # 6. Check for Scientist output data - Route to Interpreter
    elif scientist_output_data is not None and not interpreter_finished:
        print("Supervisor: Scientist code generation complete. Routing to HumanInterpreter for compilation.")
        next_agent = "HumanInterpreter"
        new_message_content = "Scientist code generation complete. Compiling final script."
        updated_state_for_return["processed_train_name"] = state.get("processed_train_name")
        updated_state_for_return["processed_test_name"] = state.get("processed_test_name")
        print(f"Supervisor propagating names from state for next step: train='{updated_state_for_return.get('processed_train_name')}', test='{updated_state_for_return.get('processed_test_name')}'")
    
    # 7. Check for original DataAnalyst output - Route to DataScientist
    elif analyst_orig_output is not None and scientist_output_data is None:
        print("Supervisor: Original Data Analyst successfully generated code and data names. Routing to Data Scientist.")
        updated_state_for_return["processed_train_name"] = analyst_orig_output.get('processed_train_name')
        updated_state_for_return["processed_test_name"] = analyst_orig_output.get('processed_test_name')
        print(f"Supervisor propagating names from original analyst output to Scientist: train='{updated_state_for_return.get('processed_train_name')}', test='{updated_state_for_return.get('processed_test_name')}'")
        task = "Build and evaluate LightGBM and XGBoost models using the processed data."
        next_agent = "DataScientist"
    
    # 8. Initial state handling
    elif 'initial_input_message' in locals() or 'initial_input_message' in globals():
        print("Supervisor: Initial human message received. Assigning initial task to Original Data Analyst.")
        next_agent = "DataAnalyst"
        new_message_content = "Okay, starting the data analysis workflow with the Data Analyst."
        task = "Perform initial EDA and preprocessing on train/test data."
    
    # 9. Unexpected state - End workflow
    else:
        print("Supervisor: Unexpected state - no initial message or prior agent output. Ending.")
        error_flag = "Workflow ended in an unexpected state."
        next_agent = "__end__"
        new_message_content = "Workflow terminated due to an unexpected state."

    # --- Prepare Return Dictionary ---
    updated_messages = list(messages)
    if new_message_content is not None:
        if not updated_messages or (hasattr(updated_messages[-1], 'content') and updated_messages[-1].content != new_message_content):
            updated_messages.append(AIMessage(content=str(new_message_content)))

    return_dict = {
        "messages": updated_messages,
        "supervisor_tasks": tasks,
        "current_task_description": task,
        "next_agent": next_agent,
        "error": error_flag,
        "final_answer_generated": updated_state_for_return.get('final_answer_generated', final_answer_generated),
        "interpreter_finished": updated_state_for_return.get('interpreter_finished', interpreter_finished),
        "initial_train_name": initial_train_name,
        "initial_test_name": initial_test_name,
        **updated_state_for_return
    }

    print(f"DEBUG: Supervisor RETURNING next_agent: '{return_dict['next_agent']}' (Type: {type(return_dict['next_agent'])})")
    print(f"DEBUG: Supervisor RETURNING final_answer_generated: {return_dict.get('final_answer_generated')}")
    print(f"DEBUG: Supervisor RETURNING interpreter_finished: {return_dict.get('interpreter_finished')}")

    return return_dict

In [25]:
def data_analyst_node(state: GraphState) -> Dict[str, Any]:
    """Data Analyst Node - Updated for DataFrame variable names."""
    print("\n--- DATA ANALYST ---")
    task = state.get('current_task_description')
    # Assume initial DFs are named 'train_df' and 'test_df' for clarity
    # Adjust these names if your actual initial variables are different (e.g., 'train', 'test')
    initial_train_name = state.get('initial_train_name')# <-- Adjust if your initial var is different
    initial_test_name =  state.get('initial_test_name') # <-- Adjust if your initial var is different

    if not task:
        error_output = {"error": "Data Analyst received no task.", "next_agent": "Supervisor"}
        print("Returning error output:", error_output)
        return error_output

    # Simulate the names of the DataFrames that the processing code will create
    processed_train_name = f"processed_{initial_train_name}" 
    processed_test_name = f"processed_{initial_test_name}"   

    print(f"Analyst expects input DataFrame variable: '{initial_train_name}'")
    print(f"Analyst expects input DataFrame variable: '{initial_test_name}'")
    print(f"Analyst will simulate creating output DataFrames: '{processed_train_name}' and '{processed_test_name}'")

    # --- Simulate Code (Make sure variable names match below) ---
    simulated_code = f"""
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Ensure correct variable names are used in the code
train = {initial_train_name}
test = {initial_test_name}  # Use the initial test name here

# === Separate target variable from train ===
target = train['Calories']
train = train.drop(columns='Calories')

# 1) Identify numerical vs. categorical columns
num_cols = train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()

print("Numerical columns:", num_cols)
print("Categorical columns:", cat_cols)

# 2) Build preprocessing pipelines
num_pipeline = Pipeline([
    ('scaler', MinMaxScaler())
])
cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# 3) KNN imputer for any remaining NaNs
imputer = KNNImputer()

# 4) Fit on train, transform both train & test
train_array = preprocessor.fit_transform(train)
train_array = imputer.fit_transform(train_array)

test_array = preprocessor.transform(test)  # Use 'test' variable name here
test_array = imputer.transform(test_array)

# 5) Recover feature names so we can build nice DataFrames
num_feats = preprocessor.named_transformers_['num'] \
               .named_steps['scaler'].feature_names_in_
cat_feats = preprocessor.named_transformers_['cat'] \
               .named_steps['onehot'].get_feature_names_out(cat_cols)
all_feats = list(num_feats) + list(cat_feats)

# Create the processed DataFrames using the intended variable names
{processed_train_name} = pd.DataFrame(train_array, columns=all_feats)
{processed_train_name}['Calories'] = target 
{processed_test_name}  = pd.DataFrame(test_array,  columns=all_feats)

print(f"Processed train shape: {{{processed_train_name}.shape}}")  # Print using the variable name
print(f"Processed test shape:  {{{processed_test_name}.shape}}")   # Print using the variable name
    """

    return{
        "analyst_output":{
            "code": simulated_code,
            # Keep nested for completeness of analyst output details
            "processed_train_name": processed_train_name,
            "processed_test_name": processed_test_name,
        },
         "current_task_description": None, # Task is complete for Analyst
    }

def process_update(update: Dict[str, Any]) -> None:
    if not isinstance(update, dict):
        print("Update is not a dictionary.")
        return

    if 'messages' in update:
        messages = update['messages']
        print("Messages from update:")
        for message in messages:
            print(message)

    if 'code' in update:
        print("Code snippet provided in update:")
        print(update['code'])

    if 'summary' in update:
        print("Summary from update:")
        print(update['summary'])

    if 'error' in update:
        print("Error encountered:")
        print(update['error'])

In [26]:
import os
from typing import Dict, Any

# Assuming GraphState class is defined elsewhere
# Assuming necessary library imports (like pandas, numpy etc.) are at the top level of your script

def data_scientist_node(state: GraphState) -> Dict[str, Any]:
    """
    Data Scientist Node - Generates modeling code (LightGBM, XGBoost)
    using processed data names provided by the DataAnalyst node via the state.
    This version DOES NOT execute the generated code.
    """
    print("\n--- DATA SCIENTIST (Code Generation Only) ---")
    task = state.get('current_task_description')
    if not task:
        print("Scientist: No task received.")
        # Return an error structure within the expected output key
        return {
            "scientist_output": {"error": "Data Scientist received no task."},
            "next_agent": "Supervisor"
            }

    # --- Get processed DataFrame names from the analyst's output in state ---
    # The analyst node puts these names in 'analyst_output' dictionary
    analyst_result = state.get('analyst_output')
    if not analyst_result or not isinstance(analyst_result, dict):
        error_msg = "Scientist Error: Analyst output not found or invalid in state."
        print(error_msg)
        return {
            "scientist_output": {"error": error_msg},
             "next_agent": "Supervisor"
             }

    # Get the variable names *as they will be created* by the analyst's code
    processed_train_name = analyst_result.get('processed_train_name')
    processed_test_name  = analyst_result.get('processed_test_name')
    # Assume the analyst's code also creates a variable named 'target'
    # This name is hardcoded based on the analyst's provided simulated code structur

    if not processed_train_name or not processed_test_name:
        error_msg = "Scientist Error: Processed DataFrame names not found within analyst output in state."
        print(error_msg)
        return {
            "scientist_output": {"error": error_msg},
             "next_agent": "Supervisor"
             }

    print(f"Scientist generating code for task: {task}")
    print(f"Will use processed train DataFrame variable: '{processed_train_name}'")
    print(f"Will use processed test DataFrame variable : '{processed_test_name}'")


    # --- Generate Code String ---
    # Use single braces {} ONLY for variables substituted NOW ({processed_train_name}, etc.)
    # Use double braces {{}} for f-strings intended for LATER execution within the generated code.
    generated_code =f'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, mean_squared_error
import lightgbm as lgb
import xgboost as xgb
import warnings
import os # Import os to create directories if needed

# --- Suppress Warnings ---
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')
warnings.filterwarnings('ignore', category=FutureWarning)

print("\\n--- Data Scientist Code Execution Started ---")

# --- Configuration ---
# These names are substituted when the string is created in the scientist node
PROCESSED_TRAIN_NAME = '{processed_train_name}'
PROCESSED_TEST_NAME  = '{processed_test_name}' # The variable name holding the target series (created by analyst code)

OUTPUT_DIR = 'model_outputs'

train = PROCESSED_TRAIN_NAME
test = PROCESSED_TEST_NAME
# --- Setup ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Ensuring output directory exists: {{OUTPUT_DIR}}") # Inner f-string: Use {{}}

# --- Data Loading ---
print(f"Loading data: train='{{PROCESSED_TRAIN_NAME}}', test='{{PROCESSED_TEST_NAME}}', target_var='{{TARGET_VARIABLE_NAME}}'")
try:
    # Attempt to load variables from the global scope where exec runs
    # These variables are assumed to have been created by the Data Analyst's code
    train = globals()[PROCESSED_TRAIN_NAME]        # Processed features for training
    test = globals()[PROCESSED_TEST_NAME]         # Processed features for testing
    target   = globals()[TARGET_VARIABLE_NAME]        # Target variable (Series)

    print(f"Loaded train shape: {{train.shape}}")
    print(f"Loaded test shape: {{test.shape}}")
    print(f"Loaded target shape: {{target.shape}}")

    # Basic sanity check: ensure number of rows match between features and target
    if train.shape[0] != target.shape[0]:
         print(f"[ERROR] Row count mismatch between training features ({{train.shape[0]}}) and target ({{target.shape[0]}}).")
         exit(1)

except KeyError as e:
    print(f"[ERROR] Required variable '{{e.args[0]}}' not found in the execution environment.")
    print("Please ensure the Data Analyst code ran successfully and created these variables with the expected names.")
    exit(1) # Exit the executed script with an error code
except Exception as e:
    print(f"[ERROR] An unexpected error occurred during data loading: {{e}}")
    exit(1) # Exit the executed script with an error code


# --- Feature/Target Split ---
# X is the processed features DataFrame
# y is the target Series
X = train
y = train['Calories']

print(f"Feature shape (X): {{X.shape}}")
print(f"Target shape (y): {{y.shape}}")

# --- Align Test Columns ---
train_cols_expected = X.columns # Get columns in order from X
X_test = test.copy() # Start with a copy of the test set

# Add missing columns with NaN
missing_cols_in_test = set(train_cols_expected) - set(X_test.columns)
for col in missing_cols_in_test:
    print(f"[Warning] Column '{{col}}' missing in test set. Adding with 0 (assuming one-hot encoded, 0 is safer than NaN).") # Changed NaN to 0, common for OHE
    X_test[col] = 0

# Select and reorder columns to match training data exactly
extra_cols_in_test = set(X_test.columns) - set(train_cols_expected)
if extra_cols_in_test:
    print(f"[Warning] Extra columns in test set that are not in train: {{extra_cols_in_test}}. Dropping them.")
    X_test = X_test.drop(columns=list(extra_cols_in_test))

X_test = X_test[train_cols_expected] # Ensure order matches train

print(f"Feature shape (X): {{X.shape}}")
print(f"Test Feature shape (X_test): {{X_test.shape}}")
print(f"Matching columns: {{list(X.columns) == list(X_test.columns)}}") # Verify column matching

# --- Train/Validation Split ---
X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"Train split shape: {{X_tr.shape}}, Validation split shape: {{X_val.shape}}")

# --- Metrics ---
def rmsle(y_true, y_pred):
    y_pred_safe = np.maximum(0, y_pred) # Ensure non-negative predictions
    epsilon = 1e-9 # Add epsilon to avoid log(0)
    return np.sqrt(mean_squared_log_error(y_true + epsilon, y_pred_safe + epsilon))

def rmse(y_true, y_pred):
     return np.sqrt(mean_squared_error(y_true, y_pred))

# --- Model Flags ---
lgbm_success = False
xgb_success = False
can_run_xgb = True # Assume XGBoost can run initially
lgb_error = None
xgb_error = None
gbm = None # Initialize model variables
bst = None

# --- LightGBM ---
print("\\n>>> Training LightGBM...")
try:
    lgb_train = lgb.Dataset(X_tr, label=y_tr)
    lgb_val   = lgb.Dataset(X_val, label=y_val, reference=lgb_train)
    lgb_params = {{ # Dictionary literal, braces are fine
        'objective':'regression_l1', 'metric':'rmsle', 'verbosity':-1,
        'n_estimators': 1000, 'learning_rate':0.05, 'feature_fraction': 0.8,
        'bagging_fraction': 0.8, 'bagging_freq': 1, 'seed': 42, 'n_jobs': -1,
        'boosting_type': 'gbdt',
    }}
    lgb_evals = {{}} # Dictionary literal, braces are fine
    callbacks = [
        lgb.log_evaluation(period=100, show_stdv=False),
        lgb.early_stopping(stopping_rounds=50, verbose=False)
    ]

    gbm = lgb.train(
        params=lgb_params, train_set=lgb_train, valid_sets=[lgb_train, lgb_val],
        valid_names=['train','val'], callbacks=callbacks, evals_result=lgb_evals
    )
    print("LightGBM training complete.")
    lgbm_success = True # Mark success

    # Plotting
    metrics_to_plot = [lgb_params['metric']] if isinstance(lgb_params['metric'], str) else lgb_params['metric']
    # Filter to metrics actually present in evals_result keys (handle multiple metrics)
    valid_metrics_to_plot = [m for m in metrics_to_plot if m in lgb_evals.get('train', {{}})]

    if valid_metrics_to_plot:
        metric_key = valid_metrics_to_plot[0] # Plot the first metric found
        plt.figure(figsize=(8, 5))
        plt.plot(lgb_evals['train'][metric_key], label=f'train {{metric_key}}')
        plt.plot(lgb_evals['val'][metric_key],   label=f'val {{metric_key}}')
        plt.title(f'LightGBM {{metric_key.upper()}}')
        plt.legend()
        plt.xlabel('Boosting Rounds')
        plt.ylabel(metric_key.upper())
        plt.grid(True)
        lgb_plot_path = os.path.join(OUTPUT_DIR, 'lgb_metric_plot.png')
        plt.savefig(lgb_plot_path)
        plt.close()
        print(f"Saved LightGBM plot to {{lgb_plot_path}}")
    else:
        print(f"[Warning] Metrics {{metrics_to_plot}} not found in LightGBM evals results keys {{list(lgb_evals.get('train', {{}}).keys())}} for plotting.")
    # Predictions & Validation
    best_iter = gbm.best_iteration if gbm.best_iteration else lgb_params.get('n_estimators', 1000) # Use best iter or total
    y_val_pred_lgb  = gbm.predict(X_val, num_iteration=best_iter)
    y_test_pred_lgb = gbm.predict(X_test, num_iteration=best_iter)
    y_test_pred_lgb_safe = np.maximum(0, y_test_pred_lgb) # Ensure non-negative
    val_rmsle_lgb = rmsle(y_val, y_val_pred_lgb)
    print(f"LightGBM Validation RMSLE = {{val_rmsle_lgb:.5f}} (using best iter: {{best_iter}})")
    lgb_pred_path = os.path.join(OUTPUT_DIR, 'lgb_test_preds.csv')
    pd.DataFrame({{'prediction': y_test_pred_lgb_safe}}).to_csv(lgb_pred_path, index=False) # Dict literal OK
    print(f"Saved LightGBM predictions to {{lgb_pred_path}}")

except Exception as e:
    lgb_error = str(e)
    print(f"[ERROR] During LightGBM training or prediction: {{e}}")

# --- XGBoost ---
print("\\n>>> Training XGBoost...")
try:
    # Check if DMatrix creation is possible
    # XGBoost requires labels for training/validation sets
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test) # Test set does not need label

except Exception as e:
    xgb_error = f"DMatrix creation failed: {{e}}"
    print(f"[ERROR] Creating XGBoost DMatrix: {{e}}")
    can_run_xgb = False # Cannot run XGBoost if DMatrix fails

if can_run_xgb:
    try:
        xgb_params = {{ # Dictionary literal, braces are fine
            'objective':'reg:squarederror', 'eval_metric':'rmse', 'eta':0.05,
            'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8, 'seed': 42
        }}
        xgb_evals = {{}} # Dictionary literal, braces are fine
        callbacks_xgb = [
            xgb.callback.EvaluationMonitor(period=100),
            xgb.callback.EarlyStopping(rounds=50, verbose=False)
        ]

        bst = xgb.train(
            params=xgb_params, dtrain=dtrain, num_boost_round=1000,
            evals=[(dtrain,'train'), (dval,'val')],
            callbacks=callbacks_xgb,
            evals_result=xgb_evals
        )
        print("XGBoost training complete.")
        xgb_success = True # Mark success

        # Plotting
        metrics_to_plot_xgb = [xgb_params['eval_metric']] if isinstance(xgb_params['eval_metric'], str) else xgb_params['eval_metric']
         # Filter to metrics actually present
        valid_metrics_to_plot_xgb = [m for m in metrics_to_plot_xgb if m in xgb_evals.get('train', {{}})]


        if valid_metrics_to_plot_xgb:
            metric_key_xgb = valid_metrics_to_plot_xgb[0] # Plot the first metric found
            plt.figure(figsize=(8, 5))
            plt.plot(xgb_evals['train'][metric_key_xgb], label=f'train {{metric_key_xgb}}')
            plt.plot(xgb_evals['val'][metric_key_xgb],   label=f'val {{metric_key_xgb}}')
            plt.title(f'XGBoost {{metric_key_xgb.upper()}}')
            plt.legend()
            plt.xlabel('Boosting Rounds')
            plt.ylabel(metric_key_xgb.upper())
            plt.grid(True)
            xgb_plot_path = os.path.join(OUTPUT_DIR, 'xgb_metric_plot.png')
            plt.savefig(xgb_plot_path)
            plt.close()
            print(f"Saved XGBoost plot to {{xgb_plot_path}}")
        else:
             print(f"[Warning] Metrics {{metrics_to_plot_xgb}} not found in XGBoost evals results keys {{list(xgb_evals.get('train', {{}}).keys())}} for plotting.")


        # Predictions & Validation
        best_iter_xgb = bst.best_iteration
        y_val_pred_xgb  = bst.predict(dval, iteration_range=(0, best_iter_xgb)) # Use best iteration range
        y_test_pred_xgb = bst.predict(dtest, iteration_range=(0, best_iter_xgb))
        y_test_pred_xgb_safe = np.maximum(0, y_test_pred_xgb) # Ensure non-negative
        val_rmsle_xgb = rmsle(y_val, y_val_pred_xgb) # Calculate RMSLE on validation set
        print(f"XGBoost Validation RMSLE = {{val_rmsle_xgb:.5f}} (using best iter: {{best_iter_xgb}})") # Report RMSLE
        xgb_pred_path = os.path.join(OUTPUT_DIR, 'xgb_test_preds.csv')
        pd.DataFrame({{'prediction': y_test_pred_xgb_safe}}).to_csv(xgb_pred_path, index=False) # Dict literal OK
        print(f"Saved XGBoost predictions to {{xgb_pred_path}}")

    except Exception as e:
        xgb_error = str(e)
        print(f"[ERROR] During XGBoost training or prediction: {{e}}")
else:
    print("Skipping XGBoost training due to DMatrix creation error.")

# --- Final Summary ---
print("\\n--- Model Training Summary ---")
if lgbm_success:
    print(f"LightGBM: Successfully trained. Validation RMSLE: {{val_rmsle_lgb:.5f}}")
    print(f"LightGBM predictions saved to {{os.path.join(OUTPUT_DIR, 'lgb_test_preds.csv')}}")
else:
if lgb_error:
     print(f"LightGBM Error: {{lgb_error}}")

if xgb_success:
    print(f"XGBoost: Successfully trained. Validation RMSLE: {{val_rmsle_xgb:.5f}}")
    print(f"XGBoost predictions saved to {{os.path.join(OUTPUT_DIR, 'xgb_test_preds.csv')}}")
elif can_run_xgb: # Only report failure if it was attempted
    print(f"XGBoost: Training failed.")

if xgb_error and can_run_xgb: # Only report specific error if attempt was made
     print(f"XGBoost Error: {{xgb_error}}")
elif not can_run_xgb and xgb_error: # Report DMatrix error if that was the cause
    print(f"XGBoost was skipped due to DMatrix creation failure: {{xgb_error}}")


print("\\n--- Data Scientist Code Execution Finished ---")
''' 
    # --- Code Generation Complete ---
    print("--- Code Generation Complete ---")

    # --- Prepare Results (Code Only) ---
    # Since we are not executing, we only return the generated code.
    # No stdout, error capture, or artifact paths from execution.
    scientist_result = {
        "code": generated_code,
        "processed_train_name": processed_train_name, # Pass the train DataFrame name
        "processed_test_name": processed_test_name,   # Pass the test DataFrame name
    }
    

    # --- Return state update ---
    return {
        "scientist_output": scientist_result,
        "next_agent": "Supervisor",
        "current_task_description": None
    }

In [27]:
def bot_to_human_interpreter(state: GraphState) -> Dict[str, Any]:
    """
    Gathers code from Analyst and Scientist outputs, blends them into a single script,
    adds explanations within the code, and formats it for copy-pasting.
    Returns an update dict with the combined code message and ROUTES TO THE REVIEW NODE.
    Does NOT set final_answer_generated=True.
    """
    print("\n--- BOT TO HUMAN INTERPRETER (Code Compiler) ---")

    # --- Robustly retrieve and validate outputs from the state ---
    # Note: This version assumes it's receiving state after Analyst/Scientist
    analyst_output_raw = state.get('analyst_llm_output')  # Check LLM output first
    scientist_output_raw = state.get('scientist_llm_output')
    
    if not analyst_output_raw:
        analyst_output_raw = state.get('analyst_output')  # Fallback to original if LLM output not found
        
    if not scientist_output_raw:
        scientist_output_raw = state.get('scientist_output')

    # Initialize variables to avoid UnboundLocalError
    analyst_output = {}
    scientist_output = {}
    
    # Ensure they are dictionaries before proceeding
    if analyst_output_raw:
        analyst_output = analyst_output_raw if isinstance(analyst_output_raw, dict) else {}
    if scientist_output_raw:
        scientist_output = scientist_output_raw if isinstance(scientist_output_raw, dict) else {}
    
    # Initialize a list to hold parts of the combined script
    combined_code_parts = []
    
    # Add an introductory comment/header to the script
    combined_code_parts.append("# Combined Data Science and Data Analyst IA's Workflow Script\n")
    combined_code_parts.append("# Generated by Analyst-IA's Agent Team\n\n")

    # Indicate if there were errors
    analyst_error = analyst_output.get('error')
    scientist_error = scientist_output.get('error')
    
    if analyst_error or scientist_error:
        combined_code_parts.append("# !!! NOTE: Errors occurred during the automated workflow stages. Review the error notes below.\n\n")

    combined_code_parts.append("# This script combines preprocessing and modeling steps.\n")
    combined_code_parts.append("# Ensure you have the necessary libraries installed (e.g., pandas, numpy, sklearn, lightgbm, xgboost, matplotlib).\n")
    combined_code_parts.append("# Make sure your initial 'train' and 'test' DataFrames (or equivalent data loading) exist before running.\n\n")

    # --- Section 1: Data Preprocessing (from Data Analyst) ---
    combined_code_parts.append("# --- 1. Data Preprocessing ---\n")
    combined_code_parts.append("# This section handles steps like identifying column types, imputation, scaling, and encoding.\n")
    combined_code_parts.append("# It prepares the raw data into processed DataFrames ready for modeling.\n\n")

    # Get the Analyst's code and error status (safe now due to checks above)
    analyst_code = analyst_output.get('code', '')

    # Safely get expected output names, handling cases where output is missing or incomplete
    processed_train_name = analyst_output.get('processed_train_name', "processed_train_name")  # Use a reasonable default
    processed_test_name = analyst_output.get('processed_test_name', "processed_test_name")    # Use a reasonable default

    if analyst_error:
        combined_code_parts.append(f"# !!! Data Analyst Error during preprocessing execution or code generation.\n")
        combined_code_parts.append(f"# !!! Error Details: {analyst_error}\n\n")
    elif not analyst_code:
        combined_code_parts.append("# Data preprocessing code was not provided by the Analyst.\n\n")
    else:
        # Append the analyst's code
        combined_code_parts.append(analyst_code.strip())
        combined_code_parts.append("\n\n")  # Add spacing after the code block
        # Add comments explaining the *expected output variables*
        combined_code_parts.append("# Expected outputs from this section (as variables in your environment):\n")
        combined_code_parts.append(f"# - {processed_train_name}: Processed training features (potentially including target)\n")
        combined_code_parts.append(f"# - {processed_test_name}: Processed test features\n\n")

    # --- Section 2: Model Training and Evaluation (from Data Scientist) ---
    combined_code_parts.append("# --- 2. Model Training and Evaluation ---\n")
    combined_code_parts.append("# This section uses the processed data to train machine learning models (LightGBM, XGBoost) and evaluate them.\n")
    combined_code_parts.append(f"# It assumes variables like '{processed_train_name}' and '{processed_test_name}' exist from the previous section.\n\n")

    # Get the Scientist's code and error status (safe now due to checks above)
    scientist_code = scientist_output.get('code', '')

    if scientist_error:
        combined_code_parts.append(f"# !!! Data Scientist Error during modeling execution or code generation.\n")
        combined_code_parts.append(f"# !!! Error Details: {scientist_error}\n\n")
        # Optionally include stdout even if there was an error, might contain clues
        if scientist_stdout:
            combined_code_parts.append("# Captured output (may contain error details):\n")
            commented_stdout = "\n".join([f"# {line}" for line in scientist_stdout.strip().split('\n')])
            combined_code_parts.append(commented_stdout + "\n\n")
    elif not scientist_code:
        combined_code_parts.append("# Model training code was not provided by the Scientist.\n\n")
    else:
        # Append the scientist's code
        combined_code_parts.append(scientist_code.strip())
        combined_code_parts.append("\n\n")  # Add spacing after the code block
        # Add comments explaining expected outputs/artifacts
        combined_code_parts.append("# Expected outputs/artifacts from this section:\n")
        combined_code_parts.append("# - Console output showing training progress and validation scores.\n")
        combined_code_parts.append("# - 'lgb_metric_plot.png', 'xgb_metric_plot.png': Plots showing model performance during training.\n")
        combined_code_parts.append("# - 'lgb_test_preds.csv', 'xgb_test_preds.csv': CSV files with predictions on the test set.\n\n")

    # --- Section 3: Summary Notes ---
    combined_code_parts.append("# --- 3. Summary Notes ---\n")
    if analyst_error or scientist_error:
        combined_code_parts.append("# NOTE: Review the error messages and generated code sections carefully.\n")
    else:
        combined_code_parts.append("# NOTE: The automated workflow appears to have completed successfully. Review the generated files and console output.\n")

    
        # Special case: if we have raw strings instead of structured output
    if isinstance(analyst_output_raw, str) or isinstance(scientist_output_raw, str):
        if isinstance(analyst_output_raw, str):
            combined_code_parts.append("# --- Analyst Output ---\n")
            combined_code_parts.append(analyst_output_raw)
            combined_code_parts.append("\n\n")
            
        if isinstance(scientist_output_raw, str):
            combined_code_parts.append("# --- Scientist Output ---\n")
            combined_code_parts.append(scientist_output_raw)
            combined_code_parts.append("\n\n")
            
        final_python_script = "".join(combined_code_parts)
        
        human_message_content = "Here is the complete Python script generated by the workflow:\n\n"
        human_message_content += "```python\n"
        human_message_content += final_python_script
        human_message_content += "```\n"
        print("Interpreter compiled code and generated message for CALOR-IA Analyst review.")
        
        return {

            "analyst_llm_output": analyst_output_raw,
            "scientist_llm_output": scientist_output_raw,
            "messages": state.get("messages", []) + [AIMessage(content=human_message_content)],
            "interpreter_finished":True,
            "next_agent": "supervisor",
        }
    else:
        if analyst_code != None:
            
            combined_code_parts.append(analyst_code)
            combined_code_parts.append("\n# --- End of Analyst Script ---")
            
        if scientist_code != None:
            
            combined_code_parts.append(scientist_code)
            combined_code_parts.append("\n# --- End of Scientist Script ---")
    
    combined_code_parts.append("\n# --- End of Script ---")
    combined_code_parts.append("\n# Copy and paste the entire block above into your environment/notebook to run the workflow!\n")

    # Combine all parts into a single string representing the full script
    final_python_script = "".join(combined_code_parts)

    # Wrap the entire script in a single markdown code block for easy copy-pasting
    human_message_content = "Here is the complete Python script generated by the workflow, combining preprocessing and modeling steps:\n\n"
    human_message_content += "```python\n"  # Start markdown code block
    human_message_content += final_python_script
    human_message_content += "```\n"        # End markdown code block

    print("Interpreter compiled code and generated message for CALOR-IA Analyst review.")

    # Prepare the state update dictionary to return
    return {
        "analyst_output": analyst_output,
        "scientist_output": scientist_output,
        "analyst_llm_output": analyst_output_raw,
        "scientist_llm_output": scientist_output_raw,
        "messages": state.get("messages", []) + [AIMessage(content=human_message_content)],
        "interpreter_finished":True,
        "next_agent": "supervisor",  # Route to the review node
    }

# "What if 🎯🚀 we build an AI Agent 🤖 who can compete for me in this Kaggle competition 🏆📊?"

In [40]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    google_api_key=secret_value_0,
    temperature = 0.3,
    max_output_tokens = 5000,          
)


# --- Define the new CALOR_IA_DATA_ANALYST_NODE ---
def CALOR_IA_DATA_ANALYST_NODE(state: GraphState) -> Dict[str, Any]:
    """
    Reviews the compiled Python script from the interpreter and generates a summary
    or concluding message for the user using an LLM.
    """
    print("\n--- CALOR-IA DATA ANALYST (Post-Compilation Review) ---")
    
    messages = state.get("messages", []) 
    analyst_output = state.get('analyst_output')
    # Try to get compiled script directly from state first
    analyst_code = analyst_output.get("code", "")
    
    # If no direct script found, try to extract from messages
    if not analyst_code:
        # Find the last message that might contain the compiled script
        last_message = messages[-1] if messages else None
        
        if last_message and isinstance(last_message, AIMessage):
            # Attempt to extract the markdown code block content
            match = re.search(r"```python\n(.*)```", last_message.content, re.DOTALL)
            if match:
                compiled_script_content = match.group(1).strip()
                print("CALOR-IA Analyst found compiled script in message.")
            else:
                print("CALOR-IA Analyst: Could not find script markdown block in last message.")
                # Try to find messages with the script data in another format
                for msg in reversed(messages):
                    if isinstance(msg, AIMessage) and "# Combined Data Science and Data Analyst" in msg.content:
                        compiled_script_content = msg.content
                        print("CALOR-IA Analyst found script content in a message without markdown.")
                        break
                
                if not compiled_script_content:
                    compiled_script_content = last_message.content.strip() if last_message.content else "No content found in last message."
        else:
            print("CALOR-IA Analyst: Last message was not an AI message or state was empty.")
            compiled_script_content = "Error: Could not retrieve compiled script content."


    
    # Store the found script for other nodes to use
    state["analyst_code"] = analyst_code
    
    # Prepare prompt for the LLM to summarize the script
    prompt_messages = [
        CALOR_IA_DATA_ANALYST_SYSINT,
        HumanMessage(content=f"Review the following generated Python script and provide the best data processing posible :\n\n```python\n{analyst_code}...\n```\n\nPlease provide executable python preprocess pipeline for kaggle.")
    ]

    print("CALOR-IA Analyst invoking LLM for summary...")
    try:
        print("CALOR-IA Analyst: LLM response.")
        llm_response = llm.invoke(prompt_messages)
        print("CALOR-IA Analyst: LLM invocation successful.")
        summary_message_content = str(llm_response.content)
        print(f"CALOR-IA Analyst IA analyst: {summary_message_content}...")
    except Exception as e:
        print(f"--- CALOR-IA Analyst Runtime Error during LLM call ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        summary_message_content = f"An error occurred while generating the final summary: {e}"
        if not state.get('error'):
             state['error'] = summary_message_content

    # Append the new summary message to the state
    updated_messages = messages + [AIMessage(content=summary_message_content)]
    print(f"CALOR-IA Analyst adding message to state: {summary_message_content[:200]}...")

    print("CALOR-IA Analyst routing to Supervisor.")
    return {
        "analyst_llm_output": summary_message_content,
        "message" : updated_messages, # Pass the script to other nodes
        "next_agent": "Supervisor"
    }

In [42]:
# --- Define the new CALOR_IA_DATA_ANALYST_NODE ---
def CALOR_IA_DATA_SCIENTIST_NODE(state: GraphState) -> Dict[str, Any]:
    """
    Reviews the compiled Python script from the interpreter and generates a summary
    or concluding message for the user using an LLM.
    """
    print("\n--- CALOR_IA_DATA_SCIENTIST (Post-Compilation Review) ---")
    
    scientist_output = state.get("scientist_output")
    
    compiled_script_content = scientist_output.get("code")
    # If not found directly, try to get from messages
    if not compiled_script_content:
        messages = state.get("messages", [])
        last_message = messages[-1] if messages else None

        if last_message and isinstance(last_message, AIMessage):
            # Attempt to extract the markdown code block content
            match = re.search(r"```python\n(.*)```", last_message.content, re.DOTALL)
            if match:
                compiled_script_content = match.group(1).strip()
                print("CALOR-IA Scientist found compiled script.")
            else:
                print("CALOR-IA Scientist: Could not find script markdown block in last message.")
                # Try to find script in other messages
                for msg in reversed(messages):
                    if isinstance(msg, AIMessage) and "# Combined Data Science and Data Analyst" in msg.content:
                        compiled_script_content = msg.content
                        print("CALOR-IA Scientist found script content in a message without markdown.")
                        break
                
                if not compiled_script_content:
                    compiled_script_content = last_message.content.strip() if last_message.content else "No content found in last message."
        else:
            print("CALOR-IA Scientist: Last message was not an AI message or state was empty.")
            compiled_script_content = "Error: Could not retrieve compiled script content."

    # Prepare prompt for the LLM to summarize the script
    prompt_messages = [
        CALOR_IA_DATA_SCIENTIST_SYSINT,
        HumanMessage(content=f"Review the following generated Python script and provide the same models and two new models:\n\n```python\n{compiled_script_content}...\n```\n\n provide two version of combine models \n\n Please provide your python code ready to run on Kaggle.")
    ]

    print("CALOR_IA_DATA_SCIENTIST...")
    try:
        print("CALOR-IA Scientist: LLM response.")
        llm_response = llm.invoke(prompt_messages)
        print("CALOR-IA Scientist: LLM invocation successful.")
        summary_message_content = str(llm_response.content)
        print(f"CALOR_IA_DATA_SCIENTIST LLM response content: {summary_message_content}...")
    except Exception as e:
        print(f"--- CALOR_IA_DATA_SCIENTIST Runtime Error during LLM call ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        summary_message_content = f"An error occurred while generating the final summary: {e}"
        if not state.get('error'):
             state['error'] = summary_message_content

    # Append the new summary message to the state
    updated_messages = state.get("messages", []) + [AIMessage(content=summary_message_content)]
    print(f"CALOR_IA_DATA_SCIENTIST adding message to state: {summary_message_content}...")

    print("CALOR-IA Scientist routing to Supervisor.")
    return {
        "scientist_llm_output": summary_message_content,
        "message": updated_messages,  # Pass the script forward
        "next_agent": "Supervisor"
    }

In [43]:


def CODE_COMPILER_IA(state: GraphState) -> Dict[str, Any]:
    """
    Combines the preprocessing script with the model training script and outputs the final result.
    """
    print("\n--- CODE_COMPILER_IA (Combining Scripts) ---")
    
    # Get preprocessing script
    preprocessing_script = [str]
    
    # Get modeling script - try to find in scientist_output
    analyst_output = state.get("analyst_llm_output")
    scientist_output = state.get("scientist_llm_output")
    preprocessing_script.append(analyst_output)
    preprocessing_script.append(scientist_output)
    
    messages = state.get("messages", [])
    
    # If we couldn't find modeling script in state, search through messages
    if not scientist_output and not analyst_output:
        # Find preprocessing script first (from analyst)
        for msg in reversed(messages):
            if isinstance(msg, AIMessage) and "```python" in msg.content and ("CALOR-IA Analyst: LLM response." or "CALOR-IA Scientist: LLM response.") in msg.content:
                match = re.search(r"```python\n(.*?)```", msg.content, re.DOTALL)
                if match:
                    preprocessing_script = match.group(1).strip()
                    print("CODE_COMPILER_IA found preprocessing script in message.")
                    break
    
    # Create the combined script
    prompt_messages = [
        CALOR_IA_CODE_COMPILER_SYSINT,
        HumanMessage(content=(f"""
        
        you have to set up python code ready to run on Kaggle:
        
        data_analyst pipeline
        {preprocessing_script[0]}.)
        data)scientist pipeline
        {preprocessing_script[1]}
    ]
    # Create the formatted final output ready to copy and paste on a kaggle cell preserve all code is crucial for this operation

    """)
                    )]
    try:
        print("CALOR-IA Scientist: LLM response.")
        llm_response = llm.invoke(prompt_messages)
        print("CALOR-IA Scientist: LLM invocation successful.")
        summary_message_content = str(llm_response.content)
        print(f"CALOR_IA_DATA_SCIENTIST LLM response content: {summary_message_content}...")
    except Exception as e:
        print(f"--- CALOR_IA_DATA_SCIENTIST Runtime Error during LLM call ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        summary_message_content = f"An error occurred while generating the final summary: {e}"
        if not state.get('error'):
            state['error'] = summary_message_content


                     
    print("CODE_COMPILER_IA: Combined script generated successfully.")
    
    # Return the complete combined script
    return {
        "messages": messages + [AIMessage(content=summary_message_content)],
        "code_final": summary_message_content,
        "final_answer_generated": True,
        "next_agent": "Supervisor"
    }

In [44]:
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("Supervisor", supervisor_node)
workflow.add_node("DataAnalyst", data_analyst_node)
workflow.add_node("DataAnalyst_AI", CALOR_IA_DATA_ANALYST_NODE)
workflow.add_node("DataSciientist_AI", CALOR_IA_DATA_SCIENTIST_NODE)
workflow.add_node("DataScientist", data_scientist_node)
workflow.add_node("HumanInterpreter", bot_to_human_interpreter) # Use this exact name
workflow.add_node("code_compiler_AI", CODE_COMPILER_IA)
# Define entry point
workflow.set_entry_point("Supervisor")

# --- ADD THESE EDGES ---
# After Analyst runs, go back to Supervisor
workflow.add_edge("DataAnalyst", "Supervisor")

workflow.add_edge("DataScientist", "Supervisor")

workflow.add_edge("HumanInterpreter", "Supervisor")

workflow.add_edge("DataAnalyst_AI", "Supervisor")

workflow.add_edge("DataSciientist_AI", "Supervisor")

workflow.add_edge("code_compiler_AI", "Supervisor")

# --- END OF ADDED EDGES ---

# Define conditional edges FROM SUPERVISOR ONLY
workflow.add_conditional_edges(
    "Supervisor",
    # Function to decide route based on supervisor's decision
    lambda state: state.get("next_agent"),
    # Mapping decision to node name
    {
        "DataAnalyst": "DataAnalyst",
        "DataScientist": "DataScientist",
        "HumanInterpreter": "HumanInterpreter", # Ensure this matches add_node name
        "DataAnalyst_AI": "DataAnalyst_AI",
        "DataScientist_AI": "DataSciientist_AI",
        "code_compiler_AI":"code_compiler_AI",
        "Supervisor": "Supervisor", # Allow looping back if needed (e.g., waiting)
        "__end__": END # Map "__end__" string to the graph's end state
    }
)

# Compile the graph
app = workflow.compile()
# --- Visualize the Graph (Optional, Unchanged) ---
'''try:
    from PIL import Image
    import io
    img_bytes = app.get_graph().draw_mermaid_png()
    img = Image.open(io.BytesIO(img_bytes))
except Exception as e:
    print(f"\nCould not generate graph visualization: {e}. (Might need `pip install pygraphviz` and graphviz system library)")

img'''

'try:\n    from PIL import Image\n    import io\n    img_bytes = app.get_graph().draw_mermaid_png()\n    img = Image.open(io.BytesIO(img_bytes))\nexcept Exception as e:\n    print(f"\nCould not generate graph visualization: {e}. (Might need `pip install pygraphviz` and graphviz system library)")\n\nimg'

In [45]:
import json # Make sure json is imported if you haven't already

# Assuming imports, node definitions, graph setup, and compilation are done above

# --- Run the Graph ---

# Initial state setup (unchanged)
initial_input_message = f""" Start working 💪
"""
initial_state = {
    "messages": [HumanMessage(content=initial_input_message)],
    "final_answer_generated": False,
    "current_task_description": "Preprocess and analyze training and test data.", 
}

print("\n--- STARTING WORKFLOW (Using DataFrame Variable Names) ---")

# Keep track of seen message contents to avoid reprinting supervisor messages repeatedly
seen_message_contents = set()


for step, event in enumerate(app.stream(initial_state, {"recursion_limit":20})):
    print(f"\n--- Workflow Step {step + 1} ---")
    for node_name, update in event.items():
        print(f"Processing update from node: {node_name}")

        # Check if the update is valid and process messages
        if update is None:
            print("Node returned None, no state update.")
            continue # Skip processing if the node returned None

        # Process messages added in this update
        if 'messages' in update:
            new_messages_in_update = [
                msg for msg in update['messages']
                if isinstance(msg, (AIMessage, HumanMessage)) and msg.content not in seen_message_contents
            ]
            for msg in new_messages_in_update:
                if isinstance(msg, AIMessage):
                    print(f"🤖 {node_name} says: {msg.content}")
                elif isinstance(msg, HumanMessage):
                    print(f"🧑‍💻 User says (via state): {msg.content}") # User messages might reappear if state is passed
                seen_message_contents.add(msg.content)


        # The HumanInterpreter output is already formatted with the code block
        # We don't need special processing here if it's added to messages
        # The loop above processing 'messages' will print it when the HumanInterpreter runs
        # If you wanted to handle HumanInterpreter output *differently* here, you could add:
        # if node_name == 'HumanInterpreter':
        #     # Access the state after the update to get the new message
        #     final_message = update.get('messages', [])[-1] if update.get('messages') else None
        #     if final_message and isinstance(final_message, AIMessage):
        #          print("\n--- FINAL REPORT ---")
        #          print(final_message.content) # Print the full markdown content
        #          print("--- END FINAL REPORT ---")


    print("\n" + "="*50 + "\n")

# After the loop finishes, the graph execution has completed (__end__ reached or limit hit)
print("\n--- Workflow Finished ---")




--- STARTING WORKFLOW (Using DataFrame Variable Names) ---

--- SUPERVISOR ---
Supervisor reviewing state. Last message type: HumanMessage
Supervisor received content:  Start working 💪

Original Analyst output available: False
Scientist output available: False
Interpreter finished signal: False
Final answer generated flag: False
Supervisor: Initial human message received. Assigning initial task to Original Data Analyst.
DEBUG: Supervisor RETURNING next_agent: 'DataAnalyst' (Type: <class 'str'>)
DEBUG: Supervisor RETURNING final_answer_generated: False
DEBUG: Supervisor RETURNING interpreter_finished: False

--- Workflow Step 1 ---
Processing update from node: Supervisor
🧑‍💻 User says (via state):  Start working 💪

🤖 Supervisor says: Okay, starting the data analysis workflow with the Data Analyst.



--- DATA ANALYST ---
Analyst expects input DataFrame variable: 'None'
Analyst expects input DataFrame variable: 'None'
Analyst will simulate creating output DataFrames: 'processed_None' an

# **Ready to Run!**

**This code is designed to be copied directly into a cell in your Kaggle notebook. Just make sure you have already loaded your initial train.csv and test.csv files into DataFrames named train and test respectively, before this code block.**

**The script will then:**

**🚀 Preprocess train and test using the defined pipelines and KNN Imputer, saving the results as processed_train_name and processed_test_name.**
**🧠 This version is ready to copy and paste so my goal was made and I am very produ of me if you read this message please leave a comment**
**📊 Now I  have to find how to make them make me win, or maybe you can do it for me**
**💾 you can make a node to execute the code the we do not have to copy and paste the code, or you can play with LLM for each agent**
**What's Next?**

****

**This workflow shows just how powerful collaborative AI agents can be in jumpstarting your machine learning projects. Go ahead, copy the code, run it, and see the magic happen! ✨ Let me know what you build next in the comments!**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor #Example model.  Change as needed.
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
train = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')

# Separate target variable from train
target = train['Calories']
train = train.drop(columns='Calories')

# 1) Identify numerical vs. categorical columns
num_cols = train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()

print("Numerical columns:", num_cols)
print("Categorical columns:", cat_cols)

# 2) Build preprocessing pipelines
num_pipeline = Pipeline([
    ('scaler', MinMaxScaler())
])
cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# 3) KNN imputer for any remaining NaNs
imputer = KNNImputer()

# 4) Fit on train, transform both train & test
train_array = preprocessor.fit_transform(train)
train_array = imputer.fit_transform(train_array)

test_array = preprocessor.transform(test)
test_array = imputer.transform(test_array)

# 5) Recover feature names
num_feats = preprocessor.named_transformers_['num'].named_steps['scaler'].feature_names_in_
cat_feats = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(cat_cols)
all_feats = list(num_feats) + list(cat_feats)

# Create the processed DataFrames
processed_train = pd.DataFrame(train_array, columns=all_feats)
processed_train['Calories'] = target
processed_test = pd.DataFrame(test_array, columns=all_feats)

print(f"Processed train shape: {processed_train.shape}")
print(f"Processed test shape: {processed_test.shape}")

#Model Training (Example using RandomForestRegressor)
X_train, X_val, y_train, y_val = train_test_split(processed_train.drop('Calories', axis=1), target, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42) #Change model here if needed.
model.fit(X_train, y_train)

#Prediction
predictions = model.predict(processed_test)
submission['Calories'] = predictions

#Feature Importance
feature_importances = pd.DataFrame({'Feature': all_feats, 'Importance': model.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances)
plt.title('Feature Importance')
plt.show()


#Save submission
submission.to_csv('submission.csv', index=False)

Numerical columns: ['id', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
Categorical columns: ['Sex']


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Processed train shape: (750000, 10)
Processed test shape: (250000, 9)
